In [9]:
import pandas as pd
import numpy as np

input_folder='/data/arpit.goel/23_MLWareRecommendation'

train_file=pd.read_csv(input_folder+'/train_MLWARE2.csv')
test_file=pd.read_csv(input_folder+'/test_MLWARE2.csv')


In [10]:
user_rating_mean=train_file.groupby('userId')['rating'].mean()
item_rating_mean=train_file.groupby('itemId')['rating'].mean()
avg_rating_mean=train_file['rating'].mean()


In [11]:
rating_user=test_file['userId'].map(user_rating_mean)
rating_item=test_file['itemId'].map(item_rating_mean)
rating_avg =avg_rating_mean



In [33]:
ratings=train_file.groupby(['userId','itemId'])['rating'].max().unstack().fillna(0)
from sklearn.decomposition import NMF
model = NMF(n_components=25, init='random', random_state=0)
model.fit_transform(ratings)
print model.reconstruction_err_

3898.9906393


In [58]:
ratings=train_file.groupby(['userId','itemId'])['rating'].max().unstack().fillna(0)

import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(ratings, k = 20)
X_pred = np.dot(np.dot(u, np.diag(s)), vt)
X_pred = pd.DataFrame(X_pred,columns=item_rating_mean.index,index=user_rating_mean.index).stack().reset_index()

In [63]:
print test_file.shape
#X_pred = pd.DataFrame(X_pred,columns=item_rating_mean.index,index=user_rating_mean.index).stack()
recommendations=pd.merge(test_file,X_pred,on=['userId','itemId'])
recommendations.columns=['ID','userId','itemId','rating']

recommendations['user_min']=recommendations['userId'].map(train_file.groupby('userId')['rating'].min())
recommendations['user_max']=recommendations['userId'].map(train_file.groupby('userId')['rating'].max())
recommendations['item_min']=recommendations['itemId'].map(train_file.groupby('itemId')['rating'].min())
recommendations['item_max']=recommendations['itemId'].map(train_file.groupby('itemId')['rating'].max())

recommendations['rating']=recommendations[['rating','user_min','item_min']].max(axis=1)
recommendations['rating']=recommendations[['rating','user_max','item_max']].min(axis=1)

print recommendations['rating'].mean(), train_file['rating'].mean()

(641015, 3)
2.68249705393 5.92869594973


In [36]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))